In [1]:
!pip install gurobipy

In [2]:
from gurobipy import *

## 問題例4 不確実性を導入

In [3]:
def order(a,b,c,d):
  return [a,b,c,d]

In [4]:
# nと初期配置を変更しなければいけない
O=order(1,3,2,4)
S=[i for i in range(1,4)]
H=[i for i in range(1,5)]
P=[i for i in range(1,5)]
T=[i for i in range(1,10)]
n1=[1,2,4,3]
n=[n1[O[0]-1],n1[O[1]-1],n1[O[2]-1],n1[O[3]-1]]

S,H,P,T,n

([1, 2, 3],
 [1, 2, 3, 4],
 [1, 2, 3, 4],
 [1, 2, 3, 4, 5, 6, 7, 8, 9],
 [1, 4, 2, 3])

In [5]:
m=Model("IP3")

Restricted license - for non-production use only - expires 2023-10-25


In [6]:
x,y={},{}
for s in S:
  for h in H:
    for p in P:
      for t in T:
        x[s,h,p,t]=m.addVar(vtype="B")

for s in S:
  for k in S:
    if s!=k:
      for t in T:
        if t!=len(T):
          y[s,k,t]=m.addVar(vtype="B")



In [7]:
for p in P:
   for t in T:
      m.addConstr(quicksum(x[s,h,p,t] for s in S for h in H)==n[p-1])

for t in range(1,len(T)):
   m.addConstr(quicksum(y[s,k,t] for s in S for k in S if k!=s)<=1)

In [8]:
for t in range(1,len(T)-1):
  m.addConstr(quicksum(y[s,k,t+1] for s in S for k in S if s!=k)<=quicksum(y[s,k,t] for s in S for k in S if k!=s))

for s in S:
  for t in range(2,len(T)+1):
    m.addConstr(quicksum(x[s,1,p,t] for p in P)<=1)

for s in S:
  for h in range(1,len(H)):
    for t in range(2,len(T)+1):
      m.addConstr(quicksum(x[s,h+1,p,t] for p in P)<=quicksum(x[s,h,p,t] for p in P))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]+quicksum(x[s,h+1,k,t] for k in P)<=1+x[s,h,p,t+1])

In [9]:
for s in S:
  for h in H:
    for p in P:
      for t in range(1,len(T)):
        m.addConstr(x[s,h,p,t]<=x[s,h,p,t+1]+quicksum(y[s,k,t] for k in S if k!=s))
        m.addConstr(x[s,h,p,t+1]<=x[s,h,p,t]+quicksum(y[k,s,t] for k in S if k!=s))

for s in S:
  for h in range(1,len(H)):
    for p in P:
      m.addConstr(quicksum(x[s,h+1,k,len(T)] for k in range(p,len(P)+1))<=quicksum(x[s,h,k,len(T)] for k in range(p,len(P)+1)))

for k in S:
   for t in range(1,len(T)-1):
    m.addConstr(quicksum(y[s,k,t] for s in S if s!=k)+quicksum(y[k,s,t+1] for s in S if s!=k)<=1)

In [10]:
m.addConstr(x[1,1,O[3],1]==1)
m.addConstr(x[1,2,O[3],1]==1)
m.addConstr(x[1,3,O[2],1]==1)
m.addConstr(x[1,4,O[1],1]==1)
m.addConstr(x[2,1,O[3],1]==1)
m.addConstr(x[2,2,O[2],1]==1)
m.addConstr(x[2,3,O[1],1]==1)
m.addConstr(x[2,4,O[0],1]==1)
m.addConstr(x[3,1,O[2],1]==1)
m.addConstr(x[3,2,O[2],1]==1)

<gurobi.Constr *Awaiting Model Update*>

In [11]:
m.setObjective(quicksum(y[s,k,t] for s in S for k in S if k!=s for t in range(1,len(T))))
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1270 rows, 480 columns and 6310 nonzeros
Model fingerprint: 0x3f78619c
Variable types: 0 continuous, 480 integer (480 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 558 rows and 201 columns
Presolve time: 0.04s
Presolved: 712 rows, 279 columns, 3309 nonzeros
Variable types: 0 continuous, 279 integer (279 binary)

Root relaxation: objective 4.088530e+00, 738 iterations, 0.03 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.08853    0  193          -    4.08853      -     -    0s
H    0     0                       6.0000000    4.08853  31.9%     -    0s
    

In [12]:
EPS=1.e-6

for (s,h,p,t) in x:
  if t==1:
    if x[s,h,p,t].X>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

  if t>=2:
    if (x[s,h,p,t].X-x[s,h,p,t-1].X)>EPS:
      print("x[%2s,%2s,%2s,%2s]=%3s" %(s,h,p,t,x[s,h,p,t].X))

for (s,k,t) in y:
  if y[s,k,t].X>EPS:
    print("y[%2s,%2s,%2s]=%3s" %(s,k,t,y[s,k,t].X))
print("the number of moves", m.objVal)

x[ 1, 1, 4, 1]=1.0
x[ 1, 2, 4, 1]=1.0
x[ 1, 3, 2, 1]=1.0
x[ 1, 3, 3, 5]=1.0
x[ 1, 4, 3, 1]=1.0
x[ 1, 4, 3, 6]=1.0
x[ 2, 1, 4, 1]=1.0
x[ 2, 2, 2, 1]=1.0
x[ 2, 3, 2, 7]=1.0
x[ 2, 3, 3, 1]=1.0
x[ 2, 4, 1, 1]=1.0
x[ 2, 4, 3, 3]=1.0
x[ 3, 1, 2, 1]=1.0
x[ 3, 2, 2, 1]=1.0
x[ 3, 3, 1, 2]=1.0
x[ 3, 4, 2, 4]=1.0
y[ 1, 2, 2]=1.0
y[ 1, 3, 3]=1.0
y[ 2, 1, 4]=1.0
y[ 2, 1, 5]=1.0
y[ 2, 3, 1]=1.0
y[ 3, 2, 6]=1.0
the number of moves 6.0
